# PII Text Extractor - Pipeline no Google Colab

Este notebook permite executar o pipeline completo de extração de PII (Dados Pessoais) diretamente no Google Colab.

**Funcionalidades:**
- Clonagem do repositório do GitHub
- Instalação de dependências
- Download do dataset
- Preparação dos dados (chunking)
- Fine-tuning do modelo
- Inferência em textos
- Avaliação do modelo

**Recomendação:** Use um runtime com GPU (T4 ou superior) para treino.

## 1. Configuração do Ambiente

In [ ]:
# @title Configurações {display-mode: "form"}

# URL do repositório no GitHub
GITHUB_REPO = "https://github.com/EliMCosta/pii-text-extractor-pt.git"  # @param {type:"string"}

# Branch a ser clonada
GITHUB_BRANCH = "main"  # @param {type:"string"}

# Diretório de trabalho
WORK_DIR = "/content/pii-text-extractor-pt"

In [ ]:
# Verificar GPU disponível
!nvidia-smi

In [ ]:
# Clonar o repositório do GitHub
import os

if os.path.exists(WORK_DIR):
    print(f"Diretório {WORK_DIR} já existe. Atualizando...")
    %cd {WORK_DIR}
    !git pull origin {GITHUB_BRANCH}
else:
    !git clone --branch {GITHUB_BRANCH} {GITHUB_REPO} {WORK_DIR}
    %cd {WORK_DIR}

print(f"\nDiretório atual: {os.getcwd()}")
!ls -la

In [ ]:
# Instalar dependências usando uv (muito mais rápido que pip)
%cd {WORK_DIR}

# Instalar uv
!pip install -q uv

# Corrigir incompatibilidade torch/torchvision do Colab
!uv pip install --system --quiet torchvision --index-url https://download.pytorch.org/whl/cu121 --reinstall-package torchvision

# Instalar dependências do projeto
!uv pip install --system --quiet -r requirements.txt

print("Dependências instaladas com sucesso!")

## 2. Download do Dataset

In [ ]:
# Criar diretório de dados e clonar o dataset do Hugging Face
%cd {WORK_DIR}

!mkdir -p data
%cd data

if os.path.exists("esic-ner"):
    print("Dataset já existe. Atualizando...")
    %cd esic-ner
    !git pull
    %cd ..
else:
    !git clone https://huggingface.co/datasets/EliMC/esic-ner

%cd {WORK_DIR}
print("\nArquivos do dataset:")
!ls -la data/esic-ner/

## 3. Preparação dos Dados (Smart Chunking)

In [ ]:
# Processar o dataset com Smart Chunking
%cd {WORK_DIR}

!python data_preprocessing/build_finetune_jsonl.py \
    --input data/esic-ner/train.jsonl \
    --output data/esic-ner/train_chunks.jsonl \
    --max_length 512 \
    --stride 64

print("\nArquivo processado:")
!ls -lh data/esic-ner/train_chunks.jsonl
!wc -l data/esic-ner/train_chunks.jsonl

## 4. Fine-tuning do Modelo

**Atenção:** Esta etapa pode levar vários minutos dependendo do tamanho do dataset e da GPU disponível.

In [ ]:
# @title Parâmetros de Treinamento {display-mode: "form"}

MODEL_BASE = "neuralmind/bert-base-portuguese-cased"  # @param {type:"string"}
NUM_EPOCHS = 3  # @param {type:"integer"}
BATCH_SIZE = 8  # @param {type:"integer"}
LEARNING_RATE = 5e-5  # @param {type:"number"}
OUTPUT_DIR = "outputs/pii-textx-pt"  # @param {type:"string"}

In [ ]:
# Executar o fine-tuning
%cd {WORK_DIR}

!CUDA_VISIBLE_DEVICES=0 python training/finetune_pii_token_classification.py \
    --model_name_or_path {MODEL_BASE} \
    --dataset_path data/esic-ner/train_chunks.jsonl \
    --output_dir {OUTPUT_DIR} \
    --num_train_epochs {NUM_EPOCHS} \
    --per_device_train_batch_size {BATCH_SIZE} \
    --learning_rate {LEARNING_RATE} \
    --bf16

In [ ]:
# Verificar o modelo treinado
print("Arquivos do modelo treinado:")
!ls -la {OUTPUT_DIR}/best/

## 5. Inferência

Agora você pode usar o modelo treinado para extrair PII de textos.

In [ ]:
# @title Inferência em Texto {display-mode: "form"}

TEXTO_EXEMPLO = "O CPF do solicitante João Silva é 123.456.789-00. Ele mora na Rua das Flores, 123."  # @param {type:"string"}
MODEL_PATH = "outputs/pii-textx-pt/best"  # @param {type:"string"}

In [ ]:
# Executar inferência via CLI
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {MODEL_PATH} \
    infer \
    --text "{TEXTO_EXEMPLO}"

## 6. Avaliação do Modelo

In [ ]:
# Executar avaliação no dataset de teste
%cd {WORK_DIR}

# Verificar se existe arquivo de teste
import os
test_file = "data/esic-ner/test.jsonl"
if os.path.exists(test_file):
    !python infer_pii.py \
        --model_name_or_path {MODEL_PATH} \
        eval \
        --dataset_path {test_file} \
        --report_path outputs/eval_report.md
else:
    print(f"Arquivo de teste não encontrado: {test_file}")
    print("Usando uma amostra do train.jsonl para demonstração...")
    !head -100 data/esic-ner/train.jsonl > data/esic-ner/sample_test.jsonl
    !python infer_pii.py \
        --model_name_or_path {MODEL_PATH} \
        eval \
        --dataset_path data/esic-ner/sample_test.jsonl \
        --report_path outputs/eval_report.md \
        --max_rows 100

In [ ]:
# Visualizar relatório de avaliação
from IPython.display import Markdown, display

report_path = f"{WORK_DIR}/outputs/eval_report.md"
if os.path.exists(report_path):
    with open(report_path, "r", encoding="utf-8") as f:
        display(Markdown(f.read()))
else:
    print("Relatório não encontrado.")

## 7. Salvar Modelo no Google Drive (Opcional)

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copiar modelo para o Drive
DRIVE_OUTPUT_PATH = "/content/drive/MyDrive/pii-extractor-model"  # @param {type:"string"}

!mkdir -p "{DRIVE_OUTPUT_PATH}"
!cp -r {WORK_DIR}/{OUTPUT_DIR}/best/* "{DRIVE_OUTPUT_PATH}/"

print(f"Modelo salvo em: {DRIVE_OUTPUT_PATH}")
!ls -la "{DRIVE_OUTPUT_PATH}/"

## 8. Inferência em Lote (JSONL)

In [ ]:
# Criar arquivo de exemplo para inferência em lote
import json

exemplos = [
    {"text": "Meu nome é Maria da Silva e meu CPF é 987.654.321-00."},
    {"text": "O processo SEI 00400-00123456/2024-99 foi instaurado."},
    {"text": "Solicito informações sobre a Lei 12.527/2011."},
    {"text": "Entre em contato pelo e-mail joao.santos@email.com ou telefone (61) 99999-8888."},
]

input_file = f"{WORK_DIR}/data/exemplos_inferencia.jsonl"
with open(input_file, "w", encoding="utf-8") as f:
    for ex in exemplos:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print(f"Arquivo criado: {input_file}")
!cat {input_file}

In [ ]:
# Executar inferência em lote
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {MODEL_PATH} \
    infer \
    --jsonl_in data/exemplos_inferencia.jsonl \
    --jsonl_out outputs/resultados_inferencia.jsonl

print("\nResultados:")
!cat outputs/resultados_inferencia.jsonl | python -m json.tool --no-ensure-ascii

---

## Usando um Modelo Pré-treinado do Hugging Face Hub

Se você tiver um modelo já publicado no Hugging Face Hub, pode usá-lo diretamente sem precisar treinar.

In [ ]:
# @title Usar Modelo do Hugging Face Hub {display-mode: "form"}

HF_MODEL_ID = "EliMC/pii-text-extractor-pt"  # @param {type:"string"}
TEXTO_TESTE = "O contribuinte José Santos, CPF 111.222.333-44, solicita revisão."  # @param {type:"string"}

In [ ]:
# Inferência com modelo do Hub
%cd {WORK_DIR}

!python infer_pii.py \
    --model_name_or_path {HF_MODEL_ID} \
    infer \
    --text "{TEXTO_TESTE}"